In [1]:
# !pip install scikit-uplift

### Домашнее задание

In [11]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels

from catboost import CatBoostClassifier

1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [6]:
data = pd.read_csv('materials_lection_4/HW_data.csv')
data.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [7]:
data['offer'].unique()

array(['Buy One Get One', 'No Offer', 'Discount'], dtype=object)

In [8]:
data = data.rename(columns={'offer': 'treatment','conversion': 'target'})

In [9]:
data['treatment'] = data['treatment'].replace({'Buy One Get One': 1, 'Discount': 1, 'No Offer': 0})

In [10]:
print(data.shape)
data.head()

(64000, 9)


,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0


3. сделать разбиение набора данных на тренировочную и тестовую выборки

In [12]:
X = data.drop('target', axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_learn, X_val, y_lear, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [ ]:
cat_columns = ['recency', 'zip_code', 'channel']
models_results = {
    'approach': [],
    'uplift@10': [],
    'uplift@20': []
}

5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion

9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево